In [2]:
from src.components.ragchain_runner import RAGChainRunner
from src.utils.config.app_config import AppConfig
from src.utils.config.manager import get_config
from src.utils.logger import get_logger
from src.utils.pipeline.mlflow_tracker import MLflowTracker
from src.utils.pipeline.vectorstore_manager import load_vectorstore

logger = get_logger(__name__)


def main() -> None:
    cfg = AppConfig.from_files("configs/model_config.yaml", "config.yaml")

    # รับข้อมูลผู้ใช้
    user_id = input("👤 User ID (กด Enter เพื่อข้าม): ")

    # Load MLflow config
    mlflow_config = get_config("config.yaml").get("mlflow", {})
    experiment_name = mlflow_config.get("experiment_name", "default")

    with MLflowTracker(
        experiment_name=experiment_name, run_name="rag_runner"
    ) as tracker:
        # บันทึกข้อมูลผู้ใช้
        if user_id:
            tracker.log_params({"user_id": user_id})

        # Load the vectorstore first
        logger.info("🔄 Loading vectorstore...")
        vectorstore = load_vectorstore(
            cfg, data_version="latest", mlflow_tracker=tracker
        )

        rag = RAGChainRunner(cfg, mlflow_tracker=tracker, vectorstore=vectorstore)

        print("🤖 RAG Chatbot พร้อมใช้งานแล้ว! (พิมพ์ 'quit', 'exit', หรือ 'bye' เพื่อออก)")
        print("-" * 50)

        # Interactive chat loop
        while True:
            try:
                question = input("💬 คำถาม: ").strip()

                # Check for exit commands
                if question.lower() in ["quit", "exit", "bye", "ออก", "จบ", "เลิก"]:
                    print("👋 ขอบคุณที่ใช้งาน RAG Chatbot!")
                    break

                # Skip empty questions
                if not question:
                    print("⚠️  กรุณาใส่คำถาม")
                    continue

                logger.info("🧠 Thinking...")
                answer = rag.answer(question, user_id=user_id if user_id else None)
                print(f"🤖 {answer}")
                print("-" * 50)

            except KeyboardInterrupt:
                print("\n👋 ขอบคุณที่ใช้งาน RAG Chatbot!")
                break
            except Exception as e:
                logger.error(f"เกิดข้อผิดพลาด: {e}")
                print(f"❌ เกิดข้อผิดพลาด: {e}")
                print("💡 ลองถามคำถามใหม่ หรือพิมพ์ 'quit' เพื่อออก")


if __name__ == "__main__":  # pragma: no cover
    main()


2025/07/03 22:42:45 INFO mlflow.tracking.fluent: Experiment with name 'rag-chatbot-production' does not exist. Creating a new experiment.


🤖 RAG Chatbot พร้อมใช้งานแล้ว! (พิมพ์ 'quit', 'exit', หรือ 'bye' เพื่อออก)
--------------------------------------------------
🤖 สวัสดีครับ มีข้อสงสัยหรือคำถามเกี่ยวกับ Quant Offside 3X หรือการลงทุนในสินทรัพย์ดิจิทัลอย่างไรบ้างครับ? ฉันพร้อมช่วยเหลือครับ หรือถ้าคุณต้องการข้อมูลเพิ่มเติมเกี่ยวกับ Quant Trading หรือ Crypto Trading ก็สามารถถามได้เลยครับ มีอะไรให้ช่วยครับ?
--------------------------------------------------


เกิดข้อผิดพลาด: Changing param values is not allowed. Param with key='question' was already logged with value='สวัสดี' for run ID='9305099381f049ba95d9b9870ab28cf6'. Attempted logging new value 'มีคอร์สอะไรบ้าง'.


❌ เกิดข้อผิดพลาด: Changing param values is not allowed. Param with key='question' was already logged with value='สวัสดี' for run ID='9305099381f049ba95d9b9870ab28cf6'. Attempted logging new value 'มีคอร์สอะไรบ้าง'.
💡 ลองถามคำถามใหม่ หรือพิมพ์ 'quit' เพื่อออก
👋 ขอบคุณที่ใช้งาน RAG Chatbot!
